In [82]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('./results4.csv')
df.columns = ['id', 'text', 'modText', 'sentiment', 'topics', 'hashtags']

for i in range(0, len(df)):
    df['topics'][i] = df['topics'][i].replace("'',", "")
df['topics'] = df['topics'].apply(lambda x: x[1:-1].split(', '))

#apply knn on id based on topics
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['topics'].apply(lambda x: ' '.join(x)))
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(tfidf_matrix)

def get_recommendations(id, cosine_sim=cosine_sim):
    idx = df.index[df['id'] == id][0]
    distances, indices = knn.kneighbors(tfidf_matrix[idx], n_neighbors=10)
    return df.iloc[indices[0]]

get_recommendations("DFD035A3-9B62-ED11-B67D-D85ED3AE4DFE")


C:\Users\andre\AppData\Local\Temp\ipykernel_23468\773169011.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['topics'][i] = df['topics'][i].replace("'',", "")


,id,text,modText,sentiment,topics,hashtags
0,DFD035A3-9B62-ED11-B67D-D85ED3AE4DFE,"Fitch have affirmed Malta's A+ rating, lauding...",fitch affirm malta rate laud first budget admi...,0,"['Inflation', 'Fitch', 'Budget', 'Social']",[]
1254,AEF68502-AE62-ED11-B67D-D85ED3AE4DFE,🚨Revise EU long-term budget as soon as possibl...,revis long term budget soon possibl grappl exc...,0,['Budget'],['MFF']
18,DFD035A3-9B62-ED11-B67D-D85ED3AE4DFE,"Over two years, this administration will have ...",year administr spent billion euro guarante sta...,0,['Inflation'],[]
2299,02E9AD93-AD62-ED11-B67D-D85ED3AE4DFE,RT @RepAOC: The $857 billion budget that passe...,billion budget pass congress week billion high...,0,"['Budget', 'Congress']",[]
21,DFD035A3-9B62-ED11-B67D-D85ED3AE4DFE,At #EUCO we bid farewell to a great friend. Th...,#euco farewel great friend thank mario draghi ...,4,['Social'],['EUCO']
2544,B31A635D-A162-ED11-B67D-D85ED3AE4DFE,Jane and I want to wish you and yours a safe a...,jane want wish safe happi year year ahead must...,0,['Social'],[]
2641,4E8AB055-A962-ED11-B67D-D85ED3AE4DFE,But I was reliably informed that gender is a s...,reliabl inform gender social construct,0,['Social'],[]
2341,AEF68502-AE62-ED11-B67D-D85ED3AE4DFE,🎧 Your morning #podcast!\n\nIn light of high e...,morn #podcast light high energi price mani dis...,4,"['EU', 'Budget', 'Crisis']","['podcast', 'CohesionPolicy']"
1253,AEF68502-AE62-ED11-B67D-D85ED3AE4DFE,"📅 Today, @EP_Budgets will vote on a new Parlia...",today vote parliamentari resolut budget long t...,0,"['EU', 'Budget', 'Union']",['MFF']
19,DFD035A3-9B62-ED11-B67D-D85ED3AE4DFE,MALTA Ċertezza u Stabbiltà: The budget which w...,malta ertezza stabbilt budget put forward even...,4,"['Future', 'Vision', 'Budget']",[]


In [83]:
#remove "''," from topics
df['topics'] = df['topics'].apply(lambda x: [i.replace("'',", "") for i in x])

#get percentage of topics in each category
def get_percentage(df):
    topics = []
    for i in df['topics']:
        topics.extend(i)
    topics = pd.Series(topics)
    return topics.value_counts()/len(topics)

get_percentage(df)

''                0.210466
'EU'              0.014324
'Twitter'         0.013751
'Law'             0.012032
'News'            0.010504
                    ...   
'Religion'        0.000191
'Left'            0.000191
'Privacy'         0.000191
'@HouseGOP'       0.000191
 '@RihoTerras'    0.000191
Length: 1018, dtype: float64

In [85]:
userDF = pd.DataFrame(columns=['id', 'topics',])
userDF.id = df.id.unique()
#add all unique topics to userDF by id unless it is empty
for i in range(0, len(userDF)):
    userDF['topics'][i] = df[df['id'] == userDF['id'][i]]['topics'].sum()
    userDF['topics'][i] = [x for x in userDF['topics'][i] if x != "\'\'"]

#for each topic add sentiment based on mean of sentiment of the user's tweets with that topic
for i in range(0, len(userDF)):
    for j in range(0, len(userDF['topics'][i])):
        userDF['topics'][i][j] = userDF['topics'][i][j] + str((df[(df['id'] == userDF['id'][i]) & (df['topics'].apply(lambda x: userDF['topics'][i][j] in x))]['sentiment'].mean()).round(2))

#perform knn on userDF based on topics
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(userDF['topics'].apply(lambda x: ' '.join(x)))
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(tfidf_matrix)

def get_recommendations(id, cosine_sim=cosine_sim):
    idx = userDF.index[userDF['id'] == id][0]
    distances, indices = knn.kneighbors(tfidf_matrix[idx], n_neighbors=10)
    return userDF.iloc[indices[0]]

get_recommendations("AEF68502-AE62-ED11-B67D-D85ED3AE4DFE")

,id,topics
13,AEF68502-AE62-ED11-B67D-D85ED3AE4DFE,"['Climate Action'2.0, 'Progress'3.2, 'Life'1.6..."
0,DFD035A3-9B62-ED11-B67D-D85ED3AE4DFE,"['Inflation'1.0, 'Fitch'0.0, 'Budget'2.67, 'So..."
9,213B02D2-AA62-ED11-B67D-D85ED3AE4DFE,"['Dogs'4.0, 'the French Navy'0.0, 'Cannabis'0...."
4,1DB29AD5-A062-ED11-B67D-D85ED3AE4DFE,"['Economy'1.18, 'Inflation'1.54, 'Veterans'0.0..."
5,B31A635D-A162-ED11-B67D-D85ED3AE4DFE,"['Rich'1.71, 'Patents'0.0, 'Eli Lilly'0.0, 'CE..."
12,C3A88EB4-AD62-ED11-B67D-D85ED3AE4DFE,"['Democracy'1.2, 'Work'1.6, 'Veterans'2.0, 'De..."
7,57B01778-A962-ED11-B67D-D85ED3AE4DFE,"['Twitter'0.89, 'The Daily Wire'0.0, 'Life'1.3..."
3,70CBBF4A-9E62-ED11-B67D-D85ED3AE4DFE,"['Climate Change'0.0, 'Sustainability'0.0, 'Jo..."
10,84474B2C-AD62-ED11-B67D-D85ED3AE4DFE,"['@MyFDOT'3.0, 'DEP'4.0, 'Placement'4.0, 'Traf..."
11,02E9AD93-AD62-ED11-B67D-D85ED3AE4DFE,"['Deal'0.0, 'Trump'0.0, 'Help'2.0, 'Healthcare..."
